In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Define project root - CHANGE THIS TO YOUR PATH
project_root = r"C:\Users\Joseph\generative-health-models"

print(f"🏠 Project Root: {project_root}")

# Add to Python path for imports
sys.path.append(project_root)
sys.path.append(os.path.join(project_root, 'src'))

# Verify project structure exists
required_dirs = [
    "data/raw/wesad", 
    "data/processed", 
    "data/synthetic",
    "models/tc_multigan", 
    "results/figures", 
    "results/metrics",
    "src/data",
    "src/models",
    "src/evaluation"
]

print("\n🏗️ Verifying project structure:")
for dir_path in required_dirs:
    full_path = os.path.join(project_root, dir_path)
    exists = os.path.exists(full_path)
    print(f"  {'✅' if exists else '❌'} {dir_path}")
    if not exists:
        os.makedirs(full_path, exist_ok=True)
        print(f"    📁 Created: {full_path}")

print(f"\n✅ Project setup completed!")


🏠 Project Root: C:\Users\Joseph\generative-health-models

🏗️ Verifying project structure:
  ✅ data/raw/wesad
  ✅ data/processed
  ✅ data/synthetic
  ✅ models/tc_multigan
  ✅ results/figures
  ✅ results/metrics
  ✅ src/data
  ✅ src/models
  ✅ src/evaluation

✅ Project setup completed!


In [8]:
def explore_wesad_structure():
    """Explore the actual WESAD dataset structure (corrected version)"""
    
    wesad_path = os.path.join(project_root, "data", "raw", "wesad")
    
    print("\n" + "="*60)
    print("WESAD DATASET STRUCTURE ANALYSIS (CORRECTED)")
    print("="*60)
    
    # Find all pickle files
    pickle_files = []
    for root, dirs, files in os.walk(wesad_path):
        for file in files:
            if file.endswith('.pkl'):
                pickle_files.append(os.path.join(root, file))
    
    print(f"📁 Found {len(pickle_files)} subject files:")
    for i, pfile in enumerate(pickle_files[:10]):
        print(f"  {i+1:2d}. {os.path.basename(pfile)}")
    
    if len(pickle_files) > 10:
        print(f"     ... and {len(pickle_files) - 10} more files")
    
    # Load one subject to understand the actual data structure
    print(f"\n🔍 Analyzing sample subject: {os.path.basename(pickle_files[0])}")
    
    try:
        with open(pickle_files[0], 'rb') as f:
            subject_data = pickle.load(f, encoding='latin1')
        
        print(f"\n📋 Top-level data structure:")
        for key in subject_data.keys():
            data_type = type(subject_data[key])
            print(f"  - '{key}': {data_type}")
        
        # Explore 'signal' key more carefully
        if 'signal' in subject_data:
            signals = subject_data['signal']
            print(f"\n📶 Signal modalities:")
            
            for signal_key in signals.keys():
                signal_data = signals[signal_key]
                print(f"  - '{signal_key}': {type(signal_data)}")
                
                # Check if it's a dictionary (nested structure)
                if isinstance(signal_data, dict):
                    print(f"    Nested structure:")
                    for nested_key in signal_data.keys():
                        nested_data = signal_data[nested_key]
                        if hasattr(nested_data, 'shape'):
                            print(f"      • {nested_key}: {nested_data.shape} ({nested_data.dtype})")
                        else:
                            print(f"      • {nested_key}: {type(nested_data)}")
                
                # Check if it's a numpy array directly
                elif hasattr(signal_data, 'shape'):
                    print(f"    Shape: {signal_data.shape}")
                    print(f"    Data type: {signal_data.dtype}")
                    
                    if signal_key == 'chest':
                        print(f"    🫀 Chest sensor channels:")
                        print(f"       0: ECG - Electrocardiogram")
                        print(f"       1: EDA - Electrodermal Activity") 
                        print(f"       2: EMG - Electromyogram")
                        print(f"       3: Resp - Respiration")
                        print(f"       4: Temp - Temperature")
        
        # Explore 'label' key
        if 'label' in subject_data:
            labels = subject_data['label']
            print(f"\n🏷️ Labels:")
            print(f"  Type: {type(labels)}")
            
            if hasattr(labels, 'shape'):
                print(f"  Shape: {labels.shape}")
                print(f"  Data type: {labels.dtype}")
                unique_labels = np.unique(labels)
                print(f"  Unique labels: {unique_labels}")
                
                label_meanings = {
                    0: "Not defined/Transient",
                    1: "Baseline", 
                    2: "Stress",
                    3: "Amusement", 
                    4: "Meditation"
                }
                
                print(f"  Label meanings:")
                for label in unique_labels:
                    if label <= 4:
                        meaning = label_meanings.get(label, 'Unknown')
                        count = np.sum(labels == label)
                        percentage = (count / len(labels)) * 100
                        print(f"    {label}: {meaning} ({count:,} samples, {percentage:.1f}%)")
        
        return pickle_files, subject_data
        
    except Exception as e:
        print(f"❌ Error loading subject data: {e}")
        print(f"   Let's try a different approach...")
        
        # Alternative: try to load and inspect the raw data
        try:
            with open(pickle_files[0], 'rb') as f:
                raw_data = pickle.load(f, encoding='latin1')
            
            print(f"\n🔧 Raw data inspection:")
            print(f"   Type: {type(raw_data)}")
            
            if isinstance(raw_data, dict):
                print(f"   Keys: {list(raw_data.keys())}")
                
                # Try to navigate the structure manually
                for key, value in raw_data.items():
                    print(f"   {key}: {type(value)}")
                    if isinstance(value, dict) and len(value) < 20:
                        for subkey, subvalue in value.items():
                            print(f"     {subkey}: {type(subvalue)}")
                            if hasattr(subvalue, 'shape'):
                                print(f"       Shape: {subvalue.shape}")
            
            return pickle_files, raw_data
            
        except Exception as e2:
            print(f"❌ Alternative approach also failed: {e2}")
            return pickle_files, None

# Explore dataset structure
subject_files = explore_wesad_structure()


WESAD DATASET STRUCTURE ANALYSIS (CORRECTED)
📁 Found 15 subject files:
   1. S10.pkl
   2. S11.pkl
   3. S13.pkl
   4. S14.pkl
   5. S15.pkl
   6. S16.pkl
   7. S17.pkl
   8. S2.pkl
   9. S3.pkl
  10. S4.pkl
     ... and 5 more files

🔍 Analyzing sample subject: S10.pkl

📋 Top-level data structure:
  - 'signal': <class 'dict'>
  - 'label': <class 'numpy.ndarray'>
  - 'subject': <class 'str'>

📶 Signal modalities:
  - 'chest': <class 'dict'>
    Nested structure:
      • ACC: (3847200, 3) (float64)
      • ECG: (3847200, 1) (float64)
      • EMG: (3847200, 1) (float64)
      • EDA: (3847200, 1) (float64)
      • Temp: (3847200, 1) (float32)
      • Resp: (3847200, 1) (float64)
  - 'wrist': <class 'dict'>
    Nested structure:
      • ACC: (175872, 3) (float64)
      • BVP: (351744, 1) (float64)
      • EDA: (21984, 1) (float64)
      • TEMP: (21984, 1) (float64)

🏷️ Labels:
  Type: <class 'numpy.ndarray'>
  Shape: (3847200,)
  Data type: int32
  Unique labels: [0 1 2 3 4 5 6 7]
  Label